In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
%matplotlib inline

In [4]:
import sys

import networkx as nx


import ipywidgets as widgets
import os, shutil, shlex
import json, io
from subprocess import Popen, PIPE, STDOUT
from contextlib import redirect_stdout, redirect_stderr
from datetime import datetime
import pandas
import matplotlib as mpl
from matplotlib import pyplot
from ipywidgets.widgets.interaction import show_inline_matplotlib_plots
import glob
from collections import OrderedDict
import numpy

In [5]:
from gahyparopt.gahyperopt import *
from gahyparopt.GAUtilities import *
from gahyparopt.parameters import *

2023-07-05 09:45:58.146888: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-05 09:45:58.235318: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [6]:
# Only if multiple GPUs available
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '6'

In [7]:
class Model(object):
    generation_performance = {}
    population=None
    generations={}
    index=0
    ga=None
    
    def setup_ga(self):
        self.ga = GADriver(
            layer_counts=HIDDEN_LAYER_COUNT,
            no_neurons=HIDDEN_LAYER_NEURONS,
            rates=HIDDEN_LAYER_RATE,
            activations=HIDDEN_LAYER_ACTIVATIONS,
            layer_types=HIDDEN_LAYER_TYPE,
            optimizers=MODEL_OPTIMIZER,
            population_size=self.population_size,
            best_candidates_count=max(1, self.population_size//3),
            random_candidates_count=self.population_size//3,
            optimizer_mutation_probability=OPTIMIZER_MUTATION_PROBABILITY,
            layer_mutation_probability=HIDDEN_LAYER_MUTATION_PROBABILITY,
            number_of_epochs=MODEL_EPOCHS,
            steps_per_epoch=MODEL_STEPS_PER_EPOCH
        )
        
    def load_population(self):
        json_fnames = glob.glob("*.json")
        population = []
        names = []

        for j in json_fnames:
            name = j.split(".")[0]
            names.append(name)
            try:
                individuum = read_chromosome(j.split('.')[0])
                population.append(individuum)
            except:
                print(j)
                raise

        population = OrderedDict(zip(names, population))
        for name, pop in population.items():
            if pop.accuracy is None:
                population[name].accuracy = 0.0
        
        self.population=OrderedDict(sorted(population.items(),key=lambda x: x[1].accuracy, reverse=True))
        
        self.population_size = len(self.population)
        
        
    def setup_plots(self):
        cmap = pyplot.cm.nipy_spectral  # define the colormap
        # extract all colors from the .jet map
        cmaplist = [cmap(i) for i in range(cmap.N)]

        # create the new map
        segmented_cmap = mpl.colors.LinearSegmentedColormap.from_list(
            'Custom cmap', cmaplist, self.population_size)

        segmented_cmap_list = [segmented_cmap(i) for i in range(segmented_cmap.N)]

        players = model.population.keys()
        color_player_dict = dict(zip(segmented_cmap_list, players))
        player_color_dict = dict(zip(players, segmented_cmap_list))
        # Add "0" for 0 generation
        player_color_dict['0'] = "k"
        
        self.player_color_dict = player_color_dict
        self.color_player_dict = color_player_dict


In [8]:
def get_accuracy_plot(df):
    fig, ax = pyplot.subplots(2,1,sharex=True,figsize=(7,8))
    mn = df.accuracy.min()*0.9
    mx = min(df.accuracy.max()*1.1, 1.0)
    bar_width = 0.8
    df = df.sort_values('accuracy', ascending=True)
    color = dict([(model.player_color_dict[name], name)  for name in df['name'].values])
    df.plot.barh(x='name',
                                 y='accuracy',
                                 ax=ax[0],
                                 grid=True,
                                 color=color,
                                 )
    df['accuracy'].plot.hist(ax=ax[1],
                             grid=True,
                             bins=model.population_size,
                             range=(mn,mx),
                             rwidth=bar_width,
                            )
    ax[1].set_xlim([mn,mx])
   
    return fig, ax


In [9]:
# Build graph
def get_tree(model, max_generation):
    
    graph = nx.DiGraph()
    graph.add_node("X-0", layer=0, player="0")
    
    for g,generation in model.generations.items():
        
        if g > max_generation:
            break
        for player, chrm in generation.items():
            graph.add_node("{}-{}".format(chrm.id, g), layer=g, player=player)

    edges = []

    for g,generation in model.generations.items():
        if g > max_generation:
            break
            
        for chrm in generation.values():
            parent_a = chrm.parent_a
            parent_b = chrm.parent_b
            id = chrm.id

            # if parent_a is None and parent_b is None:
            #     parent_a = id
            #     parent_b = id

            if g == 1:
                parent_a = "X"
                parent_b = "X"

            parent_a = "{}-{}".format(parent_a, g-1) 
            parent_b = "{}-{}".format(parent_b, g-1) 

            # if parent_a not in graph.nodes:
            #     parent_a = "{}-{}".format(id, g-1)
            # if parent_b not in graph.nodes:
            #     parent_b = "{}-{}".format(id, g-1)

            id = "{}-{}".format(id, g)

            edges.append((parent_a, id))
            edges.append((parent_b, id))


    graph.add_edges_from(edges)

    nodes_to_remove = []
    for node,data in graph.nodes(data=True):
        if "layer" not in data.keys():
            print("layer not in data.keys() in node {}".format(str(node)))
            nodes_to_remove.append(node)

    graph.remove_nodes_from(nodes_to_remove)

    pos = nx.multipartite_layout(graph, subset_key="layer")

    return graph,pos

In [10]:
# Pull button
pull_button = widgets.Button(description="Generation aktualisieren")
def pull_button_clicked(b):
    log_widget.clear_output()
    with log_widget:
        sync_remote_to_local('all')
        
pull_button.on_click(pull_button_clicked)
#################################################################
#
# Generation count
generation_count = widgets.BoundedIntText(value=0, min=0, max=0)

def on_generation_count_change(change):
    log_widget.clear_output()
    with log_widget:
        df = model.generation_performance[change['new']]
        
    plot_accuracy_vs_name(df)
    plot_generation_tree()
    
generation_count.observe(on_generation_count_change, names=['value'])    
#################################################################
#
# Generation load button
generation_button = widgets.Button(description="Generation laden")

def generation_button_clicked(b):
    log_widget.clear_output()
    with log_widget:
        model.load_population()
        

        df = pandas.DataFrame([
            {'name': key,
             'loss': model.population[key].loss,
             'accuracy': model.population[key].accuracy
            } for key in model.population.keys()
        ])
        model.index = model.index + 1
        model.generation_performance.update({model.index: df})
        model.generations.update({model.index: model.population})

        if model.index == 1:
            model.setup_plots()
       
        generation_count.max = model.index
        generation_count.min = 1
        generation_count.value = model.index
    
generation_button.on_click(generation_button_clicked)
#################################################################
#
# Evolve button
evolve_button = widgets.Button(description="Evolution!")
def evolve_button_clicked(b):
    log_widget.clear_output()
    with log_widget:
        new_chromosomes = model.ga.evolve_population(list(model.population.values()))
        new_chromosome_ids = [chromosome.id for chromosome in new_chromosomes]
        new_chromosome_dict = OrderedDict(zip(new_chromosome_ids, new_chromosomes))
        
        new_population = OrderedDict()
        
        old_keys = [key for key in new_chromosome_dict.keys() if key in model.population.keys()]
        new_keys = [key for key in new_chromosome_dict.keys() if key not in model.population.keys()]
        unpaired_keys = [key for key in model.population.keys() if key not in new_chromosome_dict.keys()]
        
        for key in old_keys:
            new_population[key] = new_chromosome_dict[key]
         
        for old_key, new_key in zip(unpaired_keys, new_keys):
            new_population[old_key] = new_chromosome_dict[new_key]
        
        model.population = new_population
        
evolve_button.on_click(evolve_button_clicked)
#################################################################
#
# Distribute button
distribute_button = widgets.Button(description="Generation verteilen")

def distribute_button_clicked(b):
    log_widget.clear_output()
    with log_widget:
        for name,individuum in model.population.items():
            print(name, individuum)
            write_chromosome(name=name,chromosome=individuum)
            sync_local_to_remote(name)
            
distribute_button.on_click(distribute_button_clicked)
#################################################################
#
# Plot widgets
accuracy_plot_widget = widgets.Output(layout={'border': '1px solid black', 'width':'50%', 'scroll':'true'})
generation_tree_widget = widgets.Output(layout={'border': '1px solid black', 'width':'50%', 'scroll':'true'})
log_widget = widgets.Output(layout={'border': '1px solid black', 'width':'80%', 'scroll':'true'})

def plot_accuracy_vs_name(df):
    accuracy_plot_widget.clear_output()
    with accuracy_plot_widget:
        
        fig, ax = get_accuracy_plot(df)

        show_inline_matplotlib_plots()
        
def plot_generation_tree():
    generation_tree_widget.clear_output()
    with generation_tree_widget:
        fig, ax = pyplot.subplots(1,1, figsize=(7,8))
        graph, pos = get_tree(model, generation_count.value)
        color = [model.player_color_dict[data["player"]] for v, data in graph.nodes(data=True)]
        _ = nx.draw(graph, pos, ax, node_color=color)       
        show_inline_matplotlib_plots()

init_button = widgets.Button(description="New Game!")
def init_button_clicked(b):
    with log_widget:
        model.load_population()
        model.setup_ga()
        
init_button.on_click(init_button_clicked) 
        
    
        
##################################################################
#
# GUI
gui = widgets.VBox(children=[
    init_button,
    widgets.HBox([pull_button, generation_button]),
    generation_count,
    widgets.HBox([
        accuracy_plot_widget,
        generation_tree_widget,
    ]),
    evolve_button,
    distribute_button,
    log_widget,
])

In [11]:
model = Model()

In [12]:
display(gui)

<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>